# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read CSV
vacation_df = pd.read_csv('../output_data/cities.csv')
#Drop NaN
vacation_df.dropna(inplace = True) 
vacation_df.head(15)


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
5,5,atuona,78,PF,1558378755,76,-9.80,-139.03,81.96,16.49
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
8,8,tiksi,95,RU,1558378756,88,71.64,128.87,14.10,15.70
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
10,10,aykhal,90,RU,1558378756,48,65.95,111.51,39.20,11.18


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]
humidity= vacation_df['Humidity']


fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
vacation_cities= vacation_df.loc[(vacation_df["Max Temp"]<=75)&(vacation_df["Max Temp"]>=60)&
                                 (vacation_df["Humidity"]<=85)&
                                 (vacation_df["Cloudiness"]<=50)]
vacation_cities

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05
68,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99
69,69,fortuna,20,ES,1558378767,77,38.18,-1.13,69.80,4.70
...,...,...,...,...,...,...,...,...,...,...
522,522,raudeberg,20,NO,1558378894,76,61.99,5.14,69.80,4.70
528,528,lyuban,39,BY,1558378625,68,52.80,28.00,67.92,12.15
533,533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=vacation_cities
hotel_df['Hotel Name']= ""
hotel_df     

<ipython-input-5-d6bcde87076b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']= ""


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41,
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93,
37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05,
68,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99,
69,69,fortuna,20,ES,1558378767,77,38.18,-1.13,69.80,4.70,
...,...,...,...,...,...,...,...,...,...,...,...
522,522,raudeberg,20,NO,1558378894,76,61.99,5.14,69.80,4.70,
528,528,lyuban,39,BY,1558378625,68,52.80,28.00,67.92,12.15,
533,533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82,
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24,


In [6]:
params={"radius": 5000,
        "types": "lodging",
        "keyword": "hotel",
        "key": g_key}

for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response=requests.get(url, params=params)
    
    json_response=response.json()
    #pprint(json_response)
    
    try:
        hotel_df.loc[index, "Hotel Name"]=json_response["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Skipped location")

hotel_df       

C:\Users\erhar\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Skipped location
Skipped location
Skipped location
Skipped location
Skipped location
Skipped location
Skipped location
Skipped location
Skipped location
Skipped location


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41,Lakeview Hotel Motel
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93,"Baza Otdykha ""Zaozer'ye"""
37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05,
68,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99,BejaParque Hotel
69,69,fortuna,20,ES,1558378767,77,38.18,-1.13,69.80,4.70,Balneario de Leana
...,...,...,...,...,...,...,...,...,...,...,...
522,522,raudeberg,20,NO,1558378894,76,61.99,5.14,69.80,4.70,Ulvesund FYR
528,528,lyuban,39,BY,1558378625,68,52.80,28.00,67.92,12.15,
533,533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82,Kärppälän Rustholli
538,538,la seyne-sur-mer,0,FR,1558378897,72,43.10,5.88,69.01,2.24,"Grand Hotel des Sablettes Plage, Curio Collect..."


In [12]:
#drop NaN values
nan_value = float("NaN")
hotel_df["Hotel Name"].replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
6,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41,Lakeview Hotel Motel
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93,"Baza Otdykha ""Zaozer'ye"""
68,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99,BejaParque Hotel
69,69,fortuna,20,ES,1558378767,77,38.18,-1.13,69.80,4.70,Balneario de Leana
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Capitólio
78,78,crotone,20,IT,1558378768,67,39.09,17.12,62.60,10.29,Hotel San Giorgio
84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93,Emerald Star Hotel
93,93,antioch,40,US,1558378507,63,38.00,-121.81,64.99,10.29,Best Western Plus Delta Inn & Suites
107,107,whitehorse,40,CA,1558378774,36,60.72,-135.06,64.00,3.36,Best Western Gold Rush Inn
114,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,Karem Bay


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))